[View in Colaboratory](https://colab.research.google.com/github/sthalles/tensorflow-tutorials/blob/master/Day_1_Pre_Made_Estimators.ipynb)

# Pre-Made Estimators

In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np

The Estimator is a high-level Tensorflow API. 

![alt text](https://www.tensorflow.org/images/tensorflow_programming_environment.png)

It provides functionalities for representing a complete model.
1. Building the actual model
2. Initialization
3. Logging
3. Saving and restoring
2. Measuring
3. Testing

To write a TensorFlow program based on pre-made Estimators, you must perform the following tasks:

1.  Create one or more input functions.
2.  Define the model's feature columns.
3. Instantiate an Estimator, specifying the feature columns and various hyperparameters.
4. Train and Evaluate

## Loading data

In [0]:
def maybe_download():
    train_path = tf.keras.utils.get_file(TRAIN_URL.split('/')[-1], TRAIN_URL)
    test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL)

    return train_path, test_path


In [0]:
TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']


def load_dataset():
  train_path, test_path = maybe_download()
  train_data = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
  test_data = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
  
  train_input = train_data[CSV_COLUMN_NAMES[0:-1]]
  train_labels = train_data[CSV_COLUMN_NAMES[-1]]
  
  test_input = test_data[CSV_COLUMN_NAMES[0:-1]]
  test_labels = test_data[CSV_COLUMN_NAMES[-1]]
  
  return (train_input, train_labels), (test_input, test_labels)

In [0]:
(X_train, y_train), (X_test, y_test) = load_dataset()

## Creating Input funcions

In [0]:
def train_input_fn(features, labels, batch_size):
  """
  A function that provides input data for training as minibatches
  Return: A 'tf.data.Dataset' object: tuple (features, labels). 
          Or tuple (features, labels)
  """
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
  dataset = dataset.shuffle(buffer_size=100)
  dataset = dataset.repeat()
  dataset = dataset.batch(batch_size)
  return dataset

In [0]:
def eval_input_fn(features, labels, batch_size):
  
  features = dict(features)
  if labels is None:
    inputs = features
  else:
    inputs = (dict(features), labels)
  
  dataset = tf.data.Dataset.from_tensor_slices(inputs)
  dataset = dataset.repeat(1)
  dataset = dataset.batch(batch_size)
  return dataset

## Defining the Feature Columns

Now, we have to convert our data into **Tensors** so that our Model can use it.

![alt text](https://www.tensorflow.org/images/feature_columns/some_constructors.jpg)

**Feature Columns** define the **type of features** we are going to feed into our Models.

Since the 4 feature columns of the Iris dataset are represented as continuos values, we need to especify it when creating the feature columns.

To do that, we use the: [tf.feature_column.numeric_column()](https://www.tensorflow.org/api_docs/python/tf/feature_column/numeric_column) constructor.

  - **tf.feature_column.numeric_column()** Represents real valued or numerical features.
  
Checkout: [Feature Engineering](https://www.tensorflow.org/get_started/feature_columns)
  
 


In [0]:
# feature columns define how to use the feature data
my_feature_columns = []
for feature_column in X_train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=feature_column,
                                                            default_value=None, # Default values to result missing values
                                                            normalizer=None # Function to normalize data (after default value) 
                                                            ))

## Building the Estimator

An Estimator encapsulates all the necessary parts of a model. 

Some of the available Estimators include:

1. **BoostedTrees** Classifier/Regressor
2. **DNN Classifier**/Regressor
3. **DNNLinearCombined** Classifier/Regressor
4. **Linear** Classifier/Regressor

Checkout: [tf.estimator](https://www.tensorflow.org/api_docs/python/tf/estimator)

Your turn: Use the **tf.estimator.LinearClassifier** to classify the Iris Dataset.

Things to keep in mind.
  - Linear model is very simple, for this case, pay special attention to the **number of classes** and the **learning rate** tunning.
  - Play with different configurations of **batch size**, it can have dramatic effects on how quick the model converges.

In [71]:
classifier = tf.estimator.LinearClassifier(    
    feature_columns=my_feature_columns,
    n_classes=3,
    optimizer=tf.train.FtrlOptimizer(
      learning_rate=0.1,
      l1_regularization_strength=0.001
    ))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpgveknbc4', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4cec6a8a58>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [72]:
classifier.train(input_fn=lambda: train_input_fn(X_train, y_train, batch_size=16), 
                 steps=1000 #  Number of steps for which to train model.
                )

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpgveknbc4/model.ckpt.
INFO:tensorflow:loss = 17.577797, step = 0
INFO:tensorflow:global_step/sec: 241.053
INFO:tensorflow:loss = 5.629356, step = 100 (0.418 sec)
INFO:tensorflow:global_step/sec: 298.585
INFO:tensorflow:loss = 6.692781, step = 200 (0.334 sec)
INFO:tensorflow:global_step/sec: 279.367
INFO:tensorflow:loss = 4.3902354, step = 300 (0.361 sec)
INFO:tensorflow:global_step/sec: 250.665
INFO:tensorflow:loss = 4.652293, step = 400 (0.396 sec)
INFO:tensorflow:global_step/sec: 248.653
INFO:tensorflow:loss = 5.2302628, step = 500 (0.404 sec)
INFO:tensorflow:global_step/sec: 245.815
INFO:tensorflow:loss = 4.7758217, step = 600 (0.406 sec)
INFO:tensorflow:global_step/sec: 257.672
INFO:tensorflow:los

In [76]:
classifier.evaluate(input_fn=lambda: eval_input_fn(X_test, y_test, batch_size=512))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-09-16:28:34
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpgveknbc4/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-09-16:28:35
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.96666664, average_loss = 0.26361477, global_step = 1000, loss = 7.9084435


{'accuracy': 0.96666664,
 'average_loss': 0.26361477,
 'global_step': 1000,
 'loss': 7.9084435}

**Exercise**: Change the Linear Model Estimator to a Deep Neural Network.
- Head over to the Tensorflow documentation for [tf.estimator.DNNClassifier](https://www.tensorflow.org/api_docs/python/tf/estimator/DNNClassifier) and check it out.

Think about how many layers you need, the number of units in each layer, the activation function used by default, and the Gradient Descent Optimizer. 

![alt text](https://www.tensorflow.org/images/custom_estimators/full_network.png)